In [ ]:
import os 
import locale
from datetime import date, timedelta
import random, json

In [2]:
class Cookbook:
    '''Это класс для работы с рецептами'''
    def __init__(self, cookbook):
        with open(cookbook, "r") as read_file:
            self.cookbook = json.load(read_file)
        self.cookbook_list = {}
        for kind in self.cookbook.keys():
            self.cookbook_list[kind] = list(self.cookbook[kind].keys())              
            
    def find(self, name):
        '''Функция ищет рецепт в книге'''
        r = 0
        for kind, kushka in self.cookbook.items():
            if name in kushka.keys(): return(Kushka(name, kushka[name])); r = 1
        if r == 0: print(name + ' not found in the cookbook')
        return None

    def find_list(self, namelist):
        '''Функция ищет рецепты из списка'''
        kushka_list = []
        for name in namelist:
            r = 0
            for kind, kushka in self.cookbook.items():
                if name in kushka.keys(): kushka_list.append(Kushka(name, kushka[name])); r = 1
            if r == 0: print(name + ' not found in the cookbook')
        return kushka_list
    
    def reload_kind(self, kind):
        '''Функция перезагружает список рецептов в категории (каши, сладости и т.д.)'''
        self.cookbook_list[kind] = list(self.cookbook[kind].keys())
        return None
    
    def print_cookbook(self):
        for kind in self.cookbook.keys():
            print(kind)
            for dish in self.cookbook[kind].keys():
                print("    " + dish, end =  ' (')
                for item in self.cookbook[kind][dish].keys():
                    if item == list(self.cookbook[kind][dish].keys())[-1]: e = ''
                    else: e = ', '
                    print(item + " - " + str(self.cookbook[kind][dish][item]), end = e)
                print(')')
        return None
    
    
    def get_random(self, kind):
        '''Функция отдает случайный рецепт выбранной категории'''
        if kind in self.cookbook.keys(): 
            if self.cookbook_list[kind] == []: self.reload_kind(kind)
            name = self.cookbook_list[kind].pop(random.randrange(len(self.cookbook_list[kind])))
            return self.find(name) 
        else: print(kind + ' is not a kind')
        
                
class Ingredient:
    '''Этот класс пока нигде не используется'''
    def __init__(self, name, kind, unit = 'г'):
        self.name = name
        self.kind = kind
        self.unit = unit
    
class Day:
    '''День'''
    def __init__(self, name):
        self.name = name
        self.eatingslist = []
    
    def add_eating(self, eating):
        self.eatingslist.append(eating)
        
        
class Eating:
    '''Прием пищи'''
    def __init__(self, kind, contents = None):
        self.kind = kind
        if type(contents) is list: self.contents = contents
        elif contents is None: self.contents = []
        else: self.contents = [contents]
    
    def add_kushka(self, contents):
        self.contents.append(contents)
    
        
    
class Kushka:
    '''Блюдо'''
    def __init__(self, name, ingredients):
        self.name = name
        self.ingredients = ingredients
        
        
class Menu:
    '''Меню на все дни'''
    def __init__(self, name, people, days = None):
        self.name = name
        self.people = people
        self.days = []

    def add_day(self, day):
        '''Функция добавляет день в меню'''
        if isinstance(day, Day): self.days.append(day)
        else: print('The day is not a day')
        return None
    
    def print_menu(self):
        '''Функция печатает меню и отдает меню куском, который удобно вставлять в latex'''
        text = '\n\n\\underline{Меню}\n\n'
        for day in self.days:
            print("\n\n" + day.name)
            text += '\medskip\medskip \n' + day.name + "\n\n"
            for eating in day.eatingslist:
                print(eating.kind, end =  ": ")
                text += eating.kind + ": "
                for kushka in eating.contents: 
                    print(kushka.name,end =  "; ")
                    text += kushka.name +  "; "
                print()
                text +="\n \n "
        return text
    
    def shopping_list(self, ingredientbook):
        '''Функция печатает список покупок и отдает его для вставки в latex'''
        text = '\\underline{Список покупок}\n\n'
        shop_list = [[], [], []]
        kinds = ingredientbook.keys()
        new_list = [[] for i in range(len(kinds) + 1)]
        for day in self.days:
            for eating in day.eatingslist:
                for kushka in eating.contents:
                    for item, amount in kushka.ingredients.items():
                        if item in shop_list[0]: 
                            shop_list[1][shop_list[0].index(item)] += amount
                            shop_list[2][shop_list[0].index(item)] += 1
                        else: shop_list[0].append(item); shop_list[1].append(amount);shop_list[2].append(1)
        for i in range(len(shop_list[0])): 
            k = 0
            for j, kind in enumerate(kinds):
                if shop_list[0][i] in ingredientbook[kind]: 
                    new_list[j].append((shop_list[0][i], shop_list[1][i] * self.people, shop_list[2][i]))
                    k = 1;continue
            if k == 0: new_list[-1].append((shop_list[0][i], shop_list[1][i] * self.people, shop_list[2][i]))
        for kind in range(len(new_list)): 
            if new_list[kind] == []: continue
            print()
            text += '\medskip \n'
            if (new_list[kind][0][0] in ingredientbook['фрукты и овощи']) or (new_list[kind][0][0] in ingredientbook['сухие супы']) or (new_list[kind][0][0] in ingredientbook["шт"]): 
                item = ' шт ('
            else: item = ' г ('
            for i in range(len(new_list[kind])): 
                print(new_list[kind][i][0] + ' - ' + str(new_list[kind][i][1]))
                if new_list[kind][i][2] in [2, 3, 4, 22, 23, 24, 32, 33, 34]: raz = " раза)\n\n"
                else: raz = ' раз)\n\n'
                text += new_list[kind][i][0] + ' - ' + str(new_list[kind][i][1]) + item + str(new_list[kind][i][2]) + raz
        return new_list, text
    def to_pdf(self):
        name_template = 'template.tex'
        f = open(name_template,'w')
        test=r'''\documentclass{article}
        \usepackage[legalpaper, portrait, margin=2cm]{geometry}
        \usepackage[utf8]{inputenc}
        \usepackage[russian,english]{babel}
        \usepackage[T2A]{fontenc}
        \usepackage{multicol}
        \begin{document}
        '''
        text1 = menu.print_menu()
        a, text2 = menu.shopping_list(ingredientbook)
        f.write(test)
        f.write('\\centering{' + menu.name)
        f.write('}\\flushleft\n\\begin{multicols}{2}')
        f.write(text1)
        f.write('\\vfill\\null \columnbreak')
        f.write(text2)
        f.write('\end{multicols}\end{document}')
        f.close()
        os.system("pdflatex " + name_template)
        print ('success')
        return None
        

def get_random_day(d):
    '''Функция отдает случайное меню на день'''
    d.add_eating(Eating('завтрак', contents = [c.get_random("Каши"), c.get_random("Сладости"), 
                                               c.find("кофе"), c.find("чай")]))
    d.add_eating(Eating('обед', contents = [c.get_random("Первое"), c.get_random("Сладости"), 
                                               c.find("хлеб"), c.find("чай")]))
    d.add_eating(Eating('перекус', contents = [c.get_random("Перекусы"), c.get_random("Перекусы")]))
    d.add_eating(Eating('ужин', contents = [c.get_random("Второе"), c.get_random("Сладости"), 
                                               c.find("кетчуп"), c.find("чай")]))
    return d

def get_random_day_winter(d):
    '''Функция отдает случайное меню на день'''
    d.add_eating(Eating('завтрак', contents = [c.get_random("Каши"), c.get_random("Сладости"), 
                                               c.find("кофе"), c.find("чай")]))
    d.add_eating(Eating('ходовой перекус', contents = [c.get_random("Бутерброды"), c.get_random("Перекусы"), c.find("чай")]))
    d.add_eating(Eating('быстросуп', contents = [c.get_random("Быстросупы"), c.find("чай")]))
    d.add_eating(Eating('ужин', contents = [c.get_random("Ужин"), c.get_random("Сладости"), 
                                               c.find("кетчуп"), c.find("чай")]))
    return d

In [3]:
# Здесь список рецептов. Количество ингредиентов указано на один раз на человека.

cookbook = {}

cookbook["Каши"]= {
  "овсянная каша" : {
    "овсянка" : 100,
    "сгущенка" : 50
  },
  "пшенка с изюмом" : {
    "пшенка" : 100,
    "сгущенка" : 50,
    "изюм" : 20
  },
  "пшеничная каша" : {
    "пшеница" : 100,
    "сгущенка" : 50
  },
    "рисовая каша" : {
        "рис круглозерный" : 80,
        "сгущенка" : 50
    },
    "кукурузная каша" : {
        "кукуруза" : 80,
        "сгущенка" : 50
    },
    "кускус": {
        "кускус" : 100,
        "сгущенка" : 50
    },
    "манная каша": {
        "манка" : 100,
        "молоко" : 0.3
    },
    
} 

cookbook["Первое"] = {
    "борщ" : {
        "сушеный борщ" : 0.25,
        "сметана" : 30,
        "тушенка" : 70
    },
    "щи" : {
        "сушеные щи" : 0.25,
        "тушенка" : 70
    },
    "солянка" : {
        "сушеная солянка" : 0.25,
        "тушенка" : 70
    },
    "рыбный суп" : {
        "рыбные консервы" : 0.25,
        "плавленный сырок" : 0.25,
        "картошка" : 0.5,
        "морковка" : 0.25,
        "лук" : 0.25,
    },
    "грибной суп": {
        "сушеные грибы": 20,
        "картошка": 0.5,
        "сушеный грибной суп": 0.25
    },
    "рассольник": {
        "куриное филе": 50,
        "картошка": 0.5,
        "морковка": 0.25,
        "соленые огурцы": 50,
        "рис": 20,
        "сметана": 20
    },
    "гороховый суп": {
        "сушеный гороховый суп": 0.25,
        "колбаса": 50,
        "картошка": 0.5,
        "горох" : 20
    }
}
cookbook["Перекусы"] = {
    "орехи" : {"орехи" : 30},
    "сухофрукты" : {"сухофрукты" : 40},
    "сушеные бананы" : {"сушеные бананы": 40}
}
cookbook["Перекусы мокрые"] = {
    "бананы" : {"бананы" : 2},
    "йогурты" : {"йогурты" : 1},
    "яблоки" : {"яблоки" : 2},
}
cookbook["Сладости"] = {
        "печенье" : {"печенье" : 40},
        "конфеты" : {"конфеты" : 40},
        "пряники" : {"пряники" : 40},
        "вафли" : {"вафли" : 50},
        "козинаки" : {"козинаки" : 40},
        "шоколад" : {"шоколад" : 40},
        "бутерброды с шоколадной пастой" : {"шоколадная паста" : 30, "хлеб" : 0.25},
        "пастила" : {"пастила" : 40},
        "зефир" : {"зефир" : 40},
        "мармелад" : {"мармелад" : 40}
}

cookbook["Разное"] = {
    "кофе" : {"кофе": 20, "сгущенка": 20},
    "чай" : {"чай": 10, "сахар": 2},
    "хлеб" :{"хлеб" : 0.25},
    "кетчуп": {"кетчуп" : 2},
    "торт": {"орехи" : 25, "сгущенка": 25, "печенье": 25, "зефирка": 1}
}

cookbook["Второе"] = {
    "макароны с тушенкой": {"макароны" : 110, "тушенка": 60},
    "гречка с тушенкой" : {"гречка" : 80, "тушенка" : 60},
    "рис с колбасой" : {"рис" : 80, "колбаса" : 50},
    "пюре с карбонадом" : {"пюре сухое" : 100, "карбонад" : 50},
    "пшенка с салом" : {"пшенка" : 80, "сало" : 50},
    "макароны с сыром-косичкой" : {"макароны" : 110, "сыр-косичка" : 0.25},
    "булгур с карбонадом" : {"булгур" : 80, "карбонад" : 50}
}

In [ ]:
# Это список ингредиентов для удобства закупки еды.
ingredientbook = {
    "крупы" : ["манка", "гречка", "рис круглозерный", "рис длиннозерный", "пшенка", "пшеница",
               "овсянка", "макароны", "пюре сухое", "кукуруза", "булгур", "кускус", "кус-кус", 
               'рис', "горох", "чечевица"],
    "фрукты и овощи" : ["картошка", "лук", "морковка", "помидоры", "огурцы", "чеснок", "бананы", "яблоки", "соленые огурцы"],
    "мясное" : ["тушенка", "мясо для шашлыка", "карбонад", "колбаса", "сало", "куриное филе", "грудинка"],
    "молочное" : ["сгущенка", "молоко", "сметана", "сливки", "йогурты"],
    "сладости" : ["конфеты", "шоколад", "печенье", "козинаки", "пастила", "мармелад", "сухари",
                  "шоколадная паста", "пряники", "щербет", "вафли", "зефир"],
    "сухие супы" : ["сушеный борщ", "сушеные щи", "сушеный грибной суп", "сушеная солянка",
                   "сушеный гороховый суп", "сушеное харчо"],
    "орехи и сухофрукты" : ["изюм", "цукаты", "сушеные бананы", "сухофрукты", "орехи", "миндаль",
                           "грецкие орехи", "кедровые орехи", "орехи и сухофрукты", "курага"],
    "шт": ["сыр-косичка", "рыбные консервы", "плавленный сырок", "хлеб", "зефирка", "протеиновый батончик",
          "суп в кружке", "томатная паста"]
}